In [1]:
from docplex.mp.model import Model
from docplex.util.environment import get_environment
import docplex

import pandas as pd
import json
import os
import numpy as np
from collections import namedtuple
import re
import time

In [2]:
docplex.__version__

'2.12.182'

In [3]:
os.chdir('/project_data/data_asset')
import model_template as mt

In [4]:
df = pd.read_csv('predictions_for_DO_production.csv')

In [6]:
df.head()

CLAIM_NBR  Base Claims Cost  Handler Case Load at Allocation  \
0   MA73673       1471.983592                            430.0   
1   MA73673       1471.983592                           1315.0   
2   MA73673       1471.983592                           1360.0   
3   MA73673       1471.983592                           1210.0   
4   MA73673       1471.983592                            690.0   

   Notification Delay  Claim Complexity        Claim Type  \
0                22.0               3.0  Personal Effects   
1                22.0               3.0  Personal Effects   
2                22.0               3.0  Personal Effects   
3                22.0               3.0  Personal Effects   
4                22.0               3.0  Personal Effects   

   Days from Policy Inception to Date of Claim  Claim Handler Level  \
0                                        274.0                  1.0   
1                                        274.0                  1.0   
2                                        274.0                  1.0   
3                                        274.0                  1.0   
4                                        274.0                  2.0   

   Number of Previous Claims  Claim Handler Minus Complexity  \
0                        1.0                             0.0   
1                        1.0                             0.0   
2                        1.0                             0.0   
3                        1.0                             0.0   
4                        1.0                             0.0   

  Handler Allocation  Pred_Leakage  
0      ClaimHandler3      0.169280  
1     ClaimHandler12      0.352303  
2     ClaimHandler15      0.352303  
3      ClaimHandler7      0.352303  
4     ClaimHandler16      0.197365

In [7]:
df["Handler Case Load at Allocation"].describe()

count    2000.000000
mean     1252.750000
std       400.479562
min       430.000000
25%      1047.500000
50%      1187.500000
75%      1536.250000
max      1970.000000
Name: Handler Case Load at Allocation, dtype: float64

# Transform Data for DO: Assignment Model

In [8]:
Claim = namedtuple("Claim", ["name", "complexity", "cost", "type"])
Handler = namedtuple("Handler", ["name", "skill", "load"])

def process_data(data, claim, handler, leakage):
    # Seperate data into dataframes
    df_claims = data[claim].drop_duplicates(subset=claim[0]).reset_index(drop=True)
    df_handlers = data[handler].drop_duplicates(subset=handler[0]).reset_index(drop=True)
    df_leakage = data[[claim[0]] + [handler[0]] + leakage].reset_index(drop=True)
    
    # Pre-allocate row memeory
    handlers = [None]*df_handlers.shape[0]
    claims = [None]*df_claims.shape[0]
    
    # Convert handlers and claims to list of named tuples
    for row_idx, row in df_handlers.iterrows():
        handlers[row_idx] = Handler(*list(row))
    
    for row_idx, row in df_claims.iterrows():
        claims[row_idx] = Claim(*list(row))
    
    # Convert leakages into a dictionary(claims:dictionary(handler:leakage))
        # Convert leakages into a dictionary(claims:dictionary(handler:leakage))
    leakages = {c_i.name:{} for c_i in claims}
    for row in range(df_leakage.shape[0]): 
        leakages[df_leakage.iloc[row,:][claim[0]]][df_leakage.iloc[row,:][handler[0]]] = df_leakage.iloc[row,:][leakage[0]] 
    
    return claims, handlers, leakages

In [9]:
handlers = ['Handler Allocation', 'Claim Handler Level', 'Handler Case Load at Allocation']
claims = ['CLAIM_NBR', 'Claim Complexity', "Base Claims Cost", 'Claim Type']
leakage = ['Pred_Leakage']
df_claims, df_handlers, df_leakage = process_data(df, claims, handlers, leakage)

In [10]:
print(len(df_claims), len(df_handlers), len(df_claims)/len(df_handlers), sum([h.load for h in df_handlers]))

100 20 5.0 25055.0


# Build DO: Assignment Model

In [11]:
model = mt.ClaimsLeakageModel(handler_budget=2280)

In [12]:
model.build_model((df_claims, df_handlers, df_leakage))

# Solve and Post Process Results

In [13]:
model.solve()

In [14]:
model.model.solution._objective

20711.491041671394

# Balanced Model

In [15]:
model_balanced_base = mt.ClaimsLeakageModel(handler_budget=2280)
model_balanced = mt.ClaimsLeakageModel(max_imbalance=5, handler_budget=2280)

model_balanced_base.build_model((df_claims, df_handlers, df_leakage))
model_balanced.build_model((df_claims, df_handlers, df_leakage))

In [16]:
model_balanced_base.solve()
model_balanced.solve()

In [17]:
print(model_balanced_base.model.solution._objective, model_balanced.model.solution._objective)

20711.491041671394 36975.31997167576


# Skill Level Limit Model

In [18]:
model_sl = mt.ClaimsLeakageModel(handler_budget=2280, skill_level_diff=1)

model_sl.build_model((df_claims, df_handlers, df_leakage))
model_sl.solve()
print(model_sl.model.solution._objective)

21140.41891364554


# Skill Level Restriction

In [19]:
model_sr = mt.ClaimsLeakageModel(handler_budget=2280, skill_restriction=20)

model_sr.build_model((df_claims, df_handlers, df_leakage))
model_sr.solve()
print(model_sr.model.solution._objective)

20737.228108375697


# Generate Solution JSONs

In [20]:
solution_df = pd.DataFrame(columns=['claim_id', 'handler_id'])
regex_get_id= re.compile("'(\w+)'", re.IGNORECASE)
for index, element in enumerate(json.loads(model_balanced_base.model.solution.export_as_json_string())['CPLEXSolution']['variables']):
    try:
        claim_id, handler_id = regex_get_id.findall(element['name'])
    except:
        claim_id, _, handler_id = regex_get_id.findall(element['name'])
    solution_df.loc[index,'claim_id'] = claim_id
    solution_df.loc[index,'handler_id'] = handler_id

outputs = {}
outputs['solution'] = solution_df.to_dict()
outputs['params'] = {"run_date":time.time(),"handler_budget":570}
import json
with open('solution_base_11022021.json', 'w') as outfile:
    json.dump(outputs, outfile, separators=(',', ':'))

In [21]:
solution_df = pd.DataFrame(columns=['claim_id', 'handler_id'])
regex_get_id= re.compile("'(\w+)'", re.IGNORECASE)
for index, element in enumerate(json.loads(model_balanced.model.solution.export_as_json_string())['CPLEXSolution']['variables']):
    try:
        claim_id, handler_id = regex_get_id.findall(element['name'])
    except:
        claim_id, _, handler_id = regex_get_id.findall(element['name'])
    solution_df.loc[index,'claim_id'] = claim_id
    solution_df.loc[index,'handler_id'] = handler_id

outputs = {}
outputs['solution'] = solution_df.to_dict()
outputs['params'] = {"run_date":time.time(),"handler_budget":570, "max_imbalance":5,}
import json
with open('solution_balanced_11022021.json', 'w') as outfile:
    json.dump(outputs, outfile, separators=(',', ':'))

In [22]:
solution_df = pd.DataFrame(columns=['claim_id', 'handler_id'])
regex_get_id= re.compile("'(\w+)'", re.IGNORECASE)
for index, element in enumerate(json.loads(model_sl.model.solution.export_as_json_string())['CPLEXSolution']['variables']):
    try:
        claim_id, handler_id = regex_get_id.findall(element['name'])
    except:
        claim_id, _, handler_id = regex_get_id.findall(element['name'])
    solution_df.loc[index,'claim_id'] = claim_id
    solution_df.loc[index,'handler_id'] = handler_id

outputs = {}
outputs['solution'] = solution_df.to_dict()
outputs['params'] = {"run_date":time.time(),"handler_budget":570, "skill_level_diff":1,}
import json
with open('solution_sll_11022021.json', 'w') as outfile:
    json.dump(outputs, outfile, separators=(',', ':'))

In [23]:
solution_df = pd.DataFrame(columns=['claim_id', 'handler_id'])
regex_get_id= re.compile("'(\w+)'", re.IGNORECASE)
for index, element in enumerate(json.loads(model_sr.model.solution.export_as_json_string())['CPLEXSolution']['variables']):
    try:
        claim_id, handler_id = regex_get_id.findall(element['name'])
    except:
        claim_id, _, handler_id = regex_get_id.findall(element['name'])
    solution_df.loc[index,'claim_id'] = claim_id
    solution_df.loc[index,'handler_id'] = handler_id

outputs = {}
outputs['solution'] = solution_df.to_dict()
outputs['params'] = {"run_date":time.time(),"handler_budget":570, "skill_restriction":20,}
import json
with open('solution_slr_11022021.json', 'w') as outfile:
    json.dump(outputs, outfile, separators=(',', ':'))

# Deploy Model

#### Note: Insert username and password below

In [20]:
from ibm_watson_machine_learning import APIClient
import os
import pandas as pd
#import numpy as np
token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5",
        "username": "insert_username", #e.g. mluser1
        "password": "insert_password" #e.g. xxxxxx
    
     }

client = APIClient(wml_credentials)

In [21]:
deployment_space_name = "ClaimsLeakage"
def guid_from_space_name(client, deployment_space_name):
    instance_details = client.service_instance.get_details()
    space = client.spaces.get_details()
    return next(item for item in space['resources'] if item['entity']['name'] == deployment_space_name)['metadata']['guid']

#### Note: insert deployment space id below under space_uid

In [22]:
project_uid = os.environ['PROJECT_ID']
space_uid = "insert_spaceid" #e.g. XXXXXX-XXXX-XXXX-XXXX-XXXXXXXX

client.set.default_project(project_uid)
client.set.default_space(space_uid)

Unsetting the project_id ...


'SUCCESS'

In [23]:
sofware_spec_uid = client.software_specifications.get_uid_by_name("do_12.10")

In [24]:
# These first few lines, makes the name of the package unique using the current time
import time
current_time = time.asctime()
metadata = {
            client.repository.ModelMetaNames.NAME: 'claims leakage DO model',
            client.repository.ModelMetaNames.TYPE: 'do-docplex_12.10',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model="/project_data/data_asset/model_template.zip",
    meta_props=metadata)

In [25]:
model_uid = client.repository.get_model_uid(published_model)

print(model_uid)

d47c41cc-5b01-4045-a9c6-4cdb101beba5


In [26]:
# Deploying the model
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "claims leakage DO model",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "Claims leakage DO model deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name" : "S", "num_nodes":2 }
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)
deployment_uid = client.deployments.get_uid(deployment_details)

print(deployment_uid)



#######################################################################################

Synchronous deployment creation for uid: 'd47c41cc-5b01-4045-a9c6-4cdb101beba5' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='9d5a2d3d-3a99-413d-9e7f-3a76956c9187'
------------------------------------------------------------------------------------------------


9d5a2d3d-3a99-413d-9e7f-3a76956c9187


# Run Model

In [27]:
client.deployments.list()

------------------------------------  -----------------------  -----  ------------------------
GUID                                  NAME                     STATE  CREATED
9d5a2d3d-3a99-413d-9e7f-3a76956c9187  claims leakage DO model  ready  2021-04-28T16:40:39.752Z
28b8d326-c5da-4145-9279-49955f5904b2  claims leakage DO model  ready  2021-04-28T16:31:19.045Z
3c6a2dcd-9ec2-4918-ad75-4978d23ffd47  claims leakage DO model  ready  2021-04-28T16:19:39.313Z
ffc683b8-04dd-4e5e-b1c7-e7e3156dc48a  claims leakage DO model  ready  2021-04-28T09:23:40.108Z
57966280-43d8-4303-894b-2f1bd099250e  Pipeline_7_webservice    ready  2021-04-20T08:27:28.044Z
------------------------------------  -----------------------  -----  ------------------------


In [28]:
# Import pandas library 
import pandas as pd 
  

In [29]:
params = pd.DataFrame([{"handler_budget":280, "max_imbalance":5}])
params

handler_budget  max_imbalance
0             280              5

In [30]:
df_claims = df[claims].drop_duplicates(subset=claims[0]).reset_index(drop=True)
df_handlers = df[handlers].drop_duplicates(subset=handlers[0]).reset_index(drop=True)
df_leakage = df[[claims[0]] + [handlers[0]] + leakage].reset_index(drop=True)

In [31]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
        {
            "id":"claim.csv",
            "values" : df_claims
        },
        {
            "id":"handler.csv",
            "values" : df_handlers
        },
        {
            "id":"leakage.csv",
            "values" : df_leakage
        },
        {
            "id":"data.csv",
            "values":df
        },
        {
            "id":"params.csv",
            "values" : params
        }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
    {
        "id":".*\.csv"
    }
    ]
}

job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print( job_uid )

d069afa2-f556-41d4-a422-9635672df242


In [32]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
queued...
queued...
queued...
queued...
running...
completed


In [33]:
job_details['entity']['decision_optimization']['status']

{'completed_at': '2021-04-28T16:41:34.749Z',
 'running_at': '2021-04-28T16:41:31.966Z',
 'state': 'completed'}

In [34]:
solution_table=[x for x in job_details['entity']['decision_optimization']['output_data'] if x['id'] == 'solution.csv'][0]

# Create a dataframe for the solution
solution = pd.DataFrame(solution_table['values'], 
                        columns = solution_table['fields'])
solution.head()

name           value
0  MA73673  ClaimHandler18
1  MA39035   ClaimHandler6
2  BI96277   ClaimHandler2
3  PE70244  ClaimHandler18
4  NO73127  ClaimHandler19

In [35]:
print( job_details['entity']['decision_optimization']['solve_state']['details']['KPI.Total Leakage Cost'] )

36975.31997167576
